# One-Hot Encoding Function

In [1]:
import numpy as np
import pandas as pd

def one_hot_encoder(data, col_name):
    
    """
    Encodes col_name according to the one-hot encoding schema.
    The new columns are named as col_name_<cat_value>.
    
    Special cases:
    - if there is no data for the transformation, the original dataframe is returned.
    - if there is missing data in the column, an exception is raised
    
    Parameters
    ----------    
    data: pandas.DataFrame
        Pandas data frame
    col_name: str
        the column name that needs to be encoded
    """
    
    #
    # checking for extreme cases
    #
    
    # if it is an empty dataframe, there is nothing to do
    if len(data) == 0:
        return(data)
        
    # getting the list of unique values
    cat_values = data[col_name].unique().tolist()
    
    # if there is missing data in the column, it is not handled yet
    try:
        if cat_values.index(None) >= 0:
            #return(data)
            raise NotImplementedError('There is missing data in the column.')
    except ValueError:
        pass

    #
    # there is some data, processing it
    #
    
    # copying data except col_name to a new data frame, it will later be returned as the result
    #df = data[data.columns.difference([col_name])].copy()
    df = data.drop(col_name, axis=1).copy()
    
    # array of the one-hot vectors, they are all zeros yet
    one_hot_vectors = np.zeros([len(data), len(cat_values)], dtype=int)

    # finding the index values for the values
    one_hot_indexes = data[col_name].map(lambda x: cat_values.index(x))

    # placing '1' to relevant places in the vectors
    one_hot_vectors[one_hot_indexes.reset_index().index, one_hot_indexes.values]  = 1

    # getting the names of new columns
    col_names = [col_name + '_' + str(value) for value in cat_values]
    
    # combining the new columns with the subset of original dataframe
    df_one_hot = pd.DataFrame(one_hot_vectors, columns=col_names)
    df_one_hot = df_one_hot.set_index(df.index)

    df = pd.concat([df, df_one_hot], axis=1)
    
    return(df)

# Testing

## Case 1: Empty Data Set

In [2]:
dfe = pd.DataFrame([], columns=['column1', 'column2'])
dfe

,column1,column2


In [3]:
one_hot_encoder(data=dfe, col_name='column1')

,column1,column2


## Case 2: No Values in Column

In [4]:
dfce = pd.DataFrame([[None, 1], [None, 2]], columns=['column1', 'column2'])
dfce

,column1,column2
0,None,1
1,None,2


In [5]:
one_hot_encoder(data=dfce, col_name='column1')

NotImplementedError: There is missing data in the column.

## Case 3: Values Exist

In [6]:
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
# https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv
dft = pd.read_csv(url).sample(10).copy()
dft

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
186,27.0,4,101.0,83.0,2202,15.3,76,europe,renault 12tl
52,30.0,4,88.0,76.0,2065,14.5,71,europe,fiat 124b
316,19.1,6,225.0,90.0,3381,18.7,80,usa,dodge aspen
274,20.3,5,131.0,103.0,2830,15.9,78,europe,audi 5000
377,31.0,4,91.0,68.0,1970,17.6,82,japan,mazda glc custom
340,25.8,4,156.0,92.0,2620,14.4,81,usa,dodge aries wagon (sw)
197,29.0,4,90.0,70.0,1937,14.2,76,europe,vw rabbit
204,32.0,4,85.0,70.0,1990,17.0,76,japan,datsun b-210
56,26.0,4,91.0,70.0,1955,20.5,71,usa,plymouth cricket
43,13.0,8,400.0,170.0,4746,12.0,71,usa,ford country squire (sw)


In [7]:
one_hot_encoder(data=dft, col_name='cylinders')

,mpg,displacement,horsepower,weight,acceleration,model_year,origin,name,cylinders_4,cylinders_6,cylinders_5,cylinders_8
186,27.0,101.0,83.0,2202,15.3,76,europe,renault 12tl,1,0,0,0
52,30.0,88.0,76.0,2065,14.5,71,europe,fiat 124b,1,0,0,0
316,19.1,225.0,90.0,3381,18.7,80,usa,dodge aspen,0,1,0,0
274,20.3,131.0,103.0,2830,15.9,78,europe,audi 5000,0,0,1,0
377,31.0,91.0,68.0,1970,17.6,82,japan,mazda glc custom,1,0,0,0
340,25.8,156.0,92.0,2620,14.4,81,usa,dodge aries wagon (sw),1,0,0,0
197,29.0,90.0,70.0,1937,14.2,76,europe,vw rabbit,1,0,0,0
204,32.0,85.0,70.0,1990,17.0,76,japan,datsun b-210,1,0,0,0
56,26.0,91.0,70.0,1955,20.5,71,usa,plymouth cricket,1,0,0,0
43,13.0,400.0,170.0,4746,12.0,71,usa,ford country squire (sw),0,0,0,1
